In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y

! pip insatll pytorch
! pip install sklearn
! pip install pandas

--2022-04-05 15:49:52--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’

Miniconda3-py37_4.8 100%[===================>]  81.12M   232MB/s    in 0.3s    

2022-04-05 15:49:52 (232 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.2’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::urllib3==1.25.8=py37_0
  - defaults/linux-64::jpeg==9d=h7f8727e_0
  - defaults/linux-64::conda-package-handling==1.6.0=py37h7b6447c_0
  - defaults/linu

## 方法：pytorch

In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error


In [3]:
data = pd.read_csv('chembl_acetylcholinesterase_bioactivity_data_processed.csv')
train_x, test_x, train_y, test_y = train_test_split(data['canonical_smiles'], data['pIC50'], test_size=0.2, random_state=1)

In [4]:
train_mols = [Chem.MolFromSmiles(smi) for smi in train_x]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
train_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in train_mols]
train_maccs_fps_array = np.asarray(train_maccs_fps, dtype=float)
train_fps_array = np.concatenate([train_morgan_fps_array, train_maccs_fps_array],axis=1)
train_x_tensor = torch.from_numpy(train_fps_array)

train_y_tensor = torch.from_numpy(np.array(train_y))


In [5]:
test_mols = [Chem.MolFromSmiles(smi) for smi in test_x]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
test_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in test_mols]
test_maccs_fps_array = np.asarray(test_maccs_fps, dtype=float)
test_fps_array = np.concatenate([test_morgan_fps_array, test_maccs_fps_array],axis=1)
test_x_tensor = torch.from_numpy(test_fps_array)

test_y_tensor = torch.from_numpy(np.array(test_y))

In [6]:
class QSAR_mlp_reg(nn.Module):
    def __init__(self):
        super(QSAR_mlp_reg, self).__init__()
        self.fc1 = nn.Linear(2215, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = F.relu(self.fc4(h3))
        return output

In [7]:
epochs_list = [50, 150, 200]
res_list = []
for epochs in epochs_list:   
  criterion = nn.MSELoss()
  model = QSAR_mlp_reg()
  #losses = []
  optimizer = optim.Adam(model.parameters(), lr=0.005)
  for epoch in range(epochs):
    data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
    optimizer.zero_grad()
    y_pred = model(data)
    #loss = F.cross_entropy(y_pred, target)
    loss = criterion(y_pred.reshape(-1), target)
    #losses.append(loss.item())
    if epoch % 20 == 0:
      print("Loss {}: {}".format(epoch, loss.item()))
    loss.backward()
    optimizer.step()
  
  pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
  pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()
  # moldel evaluation
  nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
  nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
  nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
  nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)

  nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
  nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
  nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
  nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

  nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

  result = pd.DataFrame(nn_perf, index=['train', 'test'])
  res_list.append(result)
  final_res = pd.concat(res_list, keys=epochs_list)





Loss 0: 37.66731643676758
Loss 20: 37.66731643676758
Loss 40: 37.66731643676758
Loss 0: 35.24702835083008
Loss 20: 4.692329406738281
Loss 40: 1.4500398635864258
Loss 60: 0.8605918288230896
Loss 80: 0.6056187152862549
Loss 100: 0.4461092948913574
Loss 120: 0.3433937728404999
Loss 140: 0.2767347991466522
Loss 0: 35.23900604248047
Loss 20: 1.747955083847046
Loss 40: 0.9148918986320496
Loss 60: 0.6540539860725403
Loss 80: 0.5003241896629333
Loss 100: 0.3842409551143646
Loss 120: 0.29959073662757874
Loss 140: 0.23388588428497314
Loss 160: 0.18794789910316467
Loss 180: 0.1540662944316864


In [8]:
final_res

mae    mse  mape     r2
50  train  5.97  37.67  1.00 -17.60
    test   6.00  37.95  1.00 -18.81
150 train  0.36   0.25  0.06   0.88
    test   0.60   0.77  0.11   0.60
200 train  0.24   0.13  0.04   0.94
    test   0.63   0.85  0.11   0.56